## Landsat 8 Surface Temperature Generative Script
Chase Dawson

## Part 1: Read in Spatial Data


In [169]:
# load libraries
import json
import requests
from dotenv import dotenv_values
import sys
import os
import time
import re
import geopandas as gpd
import matplotlib.pyplot as plt
import seaborn as sns
from shapely.geometry import mapping
import rioxarray as rxr
import rasterio as rio
import xarray as xr
import earthpy as et
import earthpy.plot as ep
from shapely.geometry import Polygon
from datetime import datetime
from tqdm import tqdm

%matplotlib inline

# print cwd
os.getcwd()

'/Users/chasedawson/dev/uva_equity_center/climate_equity'

In [213]:
def read_spatial_data(basename, clip_coast=False):
    """
    Reads in spatial data that is in the format 'basename_spatialUnit.shp'.
    
    Parameters
    ----------
    basename : str, required
        Base name of shape files. For example, if the files you want to read are of the format
        'cville_counties.shp', 'cville_blocks.shp', etc. then the basename is 'cville'.
        
    clip_coast : bool, optional (default is False)
        If True, shp data will be compared to coast line data and oceans will be clipped out.
    
    Output
    ------
    dictionary containing geopandas dataframes for each spatial unit
    
    """
    print('Reading spatial data for {basename}...'.format(basename = basename))
    # read in coast line data 
    if clip_coast:
        ocean = gpd.read_file('water/ne_10m_ocean.shp')
    
    # store current working directory
    og_wd = os.getcwd()
    
    # change working directory to where spatial data is located
    os.chdir("../spatial_units/data")
    
    # create empty dictionary
    data = {}
    
    if clip_coast:
        spatial_units = ['blocks', 'blkgps', 'tracts']

        # read in counties and clip first
        print("Reading counties...")
        counties = gpd.read_file(basename + '_counties.shp')
        ocean = ocean.to_crs(counties.crs)
        counties = gpd.overlay(counties, ocean, how='difference')
        data['counties'] = counties
        print("Done.")
    
        # read in rest of shapefiles, clip respective to counties, store as keys in dict
        for spatial_unit in spatial_units:
            print("Reading {spatial_unit}...".format(spatial_unit = spatial_unit))
            # read in shp file
            shp = gpd.read_file(basename + '_{spatial_unit}.shp'.format(spatial_unit = spatial_unit))

            # convert coast line data to crs of shp 
            counties = counties.to_crs(shp.crs)

            # clip out ocean
            shp = gpd.overlay(shp, counties[['geometry']], how='intersection', keep_geom_type=True)

            # add dict with spatial unit as key
            data[spatial_unit] = shp
            print("Done.")
            
    else:
        spatial_units = ['counties', 'blocks', 'blkgps', 'tracts']
        for spatial_unit in spatial_units:
            print("Reading {spatial_unit}...".format(spatial_unit = spatial_unit))
            shp = gpd.read_file(basename + '_{spatial_unit}.shp'.format(spatial_unit = spatial_unit))
            data[spatial_unit] = shp
            print("Done.")

    # reset back to original working directory
    os.chdir(og_wd)
    
    return data


In [214]:
# read in shp files
cville_shp = read_spatial_data('cville')
print('\n')
easternShore_shp = read_spatial_data('eastshore', clip_coast=True)

Reading spatial data for cville...
Reading counties...
Done.
Reading blocks...
Done.
Reading blkgps...
Done.
Reading tracts...
Done.


Reading spatial data for eastshore...
Reading counties...
Done.
Reading blocks...
Done.
Reading blkgps...
Done.
Reading tracts...
Done.


In [13]:
def get_bounds(gdf):
    """
    Get lower left and upper right (lat, lng) coordinates for all geometries
    in GeoDataFrame.
    
    Parameters
    ----------
    gdf : GeoDataFrame, required
        GeoPandas DataFrame that you want to get a bounding box for.
        
    Output
    ------
    Dictionary containing lower left and upper right (lat, lng) coordinates
    of the bounding box. 
    
    """
    bounds_df = gdf.bounds
    lowerLeft = {'latitude': round(bounds_df['miny'].min(), 6), 'longitude': round(bounds_df['minx'].min(), 6)}
    upperRight = {'latitude': round(bounds_df['maxy'].max(), 6), 'longitude': round(bounds_df['maxx'].max(), 6)}
    return {
        'lowerLeft': lowerLeft,
        'upperRight': upperRight
    }

In [17]:
# use shpfiles to get bounding boxes for Charlottesville and Eastern Shore
cville_bounds = get_bounds(cville_shp['counties'])
easternShore_bounds = get_bounds(easternShore_shp['counties'])

# get perimeter shps
cville_perimeter = cville_shp['counties'].dissolve()
easternShore_perimeter = easternShore_shp['counties'].dissolve()

## Download Landsat 8 Data


In [257]:
## Methods for working with USGS API ##

# API base URL
SERVICE_URL = "https://m2m.cr.usgs.gov/api/api/json/stable/"

def login(username, password):
    """
    Authenticates user given username and password and returns API key.
    
    Parameters
    ----------
    username : str, required
        USGS account username.
        
    password : str, required
        USGS account password. 
        
    Notes 
    -----
    Go to https://ers.cr.usgs.gov/profile/access to request access 
    to the API and/or make an account.
    
    """
    # login information
    payload = {'username': username, 'password': password}

    # get apiKey 
    apiKey = sendRequest(SERVICE_URL + "login", payload)
    if apiKey == None:
        print("Login Failed\n\n")
    else:
        print("Login Successful\n\n")
    
    return apiKey

def logout(apiKey):
    """
    Invalidates API key. 
    
    Parameters
    ----------
    apiKey : str, required
        Valid API key. Obtain using the login() method defined above.
        
    Notes
    -----
    Make sure to call when you've finished working to ensure that your 
    API key can't be used by an unauthorized user.
    
    """
    endpoint = "logout"
    if sendRequest(SERVICE_URL + endpoint, None, apiKey) == None:
        print("Logged Out\n\n")
    else:
        print("Logout Failed\n\n")

def sendRequest(url, data, apiKey = None):
    """
    Sends HTTPS request to specified API endpoint. Main method for interacting
    with the API.
    
    Parameters
    ----------
    url : str, required
        API endpoint you wish you access. Typical format is SERVICE_URL + endpoint, 
        where endpoint might be something like "login" or "data-search." See https://m2m.cr.usgs.gov/api/docs/reference/
        for all available endpoints.
        
    data : dict, required
        Request payload. Data required changes based on API endpoint. See 
        https://m2m.cr.usgs.gov/api/docs/reference/ for input parameters, sample requests,
        sample and responses for available endpoints.
        
    apiKey : str, optional (default is None)
        Valid API key. Must be speficied for most requests. "login" endpoint doesn't 
        require an API key since you use that endpoint to retrieve a valid API key.
    
    """
    json_data = json.dumps(data)
    
    if apiKey == None:
        response = requests.post(url, json_data)
    else:
        headers = {'X-Auth-Token': apiKey}
        response = requests.post(url, json_data, headers = headers)
          
    try:
        httpStatusCode = response.status_code
        
        if response == None:
            print("No output from service!")
            sys.exit()
            
        output = json.loads(response.text)
        if output['errorCode'] != None:
            print(output['errorCode'], "- ", output['errorMessage'])
            sys.exit()
            
        if httpStatusCode == 404:
            print("404 Not Found")
            sys.exit()
            
        elif httpStatusCode == 401:
            print("401 Unauthorized")
            sys.exit()
            
        elif httpStatusCode == 400:
            print("Error Code", httpStatusCode)
            sys.exit()
            
    except Exception as e:
        response.close()
        print(e)
        sys.exit()
    
    response.close()
    return output['data']

def getFilename_fromCd(cd):
    """
    Uses content-disposition to infer filename and filetype.
    
    Parameters
    ----------
    cd : str, required
        The Content-Disposition response header from HTTP request 
        to download a file.
        
    Output
    ------
    Inferred filename and type of provided file : str  
    """
    if not cd:
        return None
    fname = re.findall('filename=(.+)', cd)
    if len(fname) == 0:
        return None
    
    return re.sub('\"', '', fname[0]) # remove extra quotes

def download_file(url):
    """
    Saves file to local system.
    
    Parameters
    ----------
        url: str, required
            Link to file to be downloaded.
            
    Output
    ------
    Path to downloaded file : str
    """
    res = requests.get(url)
    filename = getFilename_fromCd(res.headers.get('content-disposition'))
    open(filename, 'wb').write(res.content)
    return filename
    
def search_scenes(apiKey, bounds, start_date, end_date, dataset = "landsat_ot_c2_l2", cloud_cover_min = 0, cloud_cover_max = 10):
    """
    Search specified dataset for scenes given spatial and temporal filters.
    
    Parameters
    ----------
    apiKey : str, required
        Valid API key.
        
    bounds: dict, required
        Dictionary with two entries: 'lowerLeft' and 'upperRight' which contain
        the lower left and upper right lat, lng coordinates of the bounding box covering
        the area of interest.
        
    start_date: str, required
        Format: YYYY-MM-DD
        
    end_date: str, required
        Format: YYYY-MM-DD
        
    dataset: str, optional (default is 'landsat_ot_c2_l2')
        Dataset alias. Use the 'dataset-search' endpoint to discover
        which datasets are available.
        
    cloud_cover_min : int, optional (default is 0)
        Minimum cloud coverage percentage. Scenes with cloud coverage less
        than this value will not be included in the result.
        
    cloud_cover_max: int, optional (default is 10)
        
    """
    payload = {
        'datasetName': dataset,
        'startingNumber': 1,
        'sceneFilter': {
            'spatialFilter': {
                'filterType': 'mbr',
                'lowerLeft': bounds['lowerLeft'],
                'upperRight': bounds['upperRight']
            },
            'acquisitionFilter': {
                'start': start_date,
                'end': end_date
            },
            'cloudCoverFilter': {
                'max': 10,
                'min': 0,
                'includeUnknown': False,
            },
            'seasonalFilter': [6,7,8,9]
        }
    }
    
    print("Searching Scenes...")
    scenes = sendRequest(SERVICE_URL + "scene-search", payload, apiKey)
    print("Found {num_scenes} Scene(s).".format(num_scenes = scenes['recordsReturned']))
    
    return scenes

def get_acquisitionDates(scenes):
    acquisitionDates = {}
    for result in scenes['results']:
        entityId = result['entityId']
        metadata = result['metadata']
        for field in metadata:
            if field['fieldName'] == "Acquisition Date":
                date = field['value']
                acquisitionDates[entityId] = date
                break
    return acquisitionDates


def get_scene_metadata(scenes):
    scene_metadata = []
    for result in scenes['results']:
        entityId = result['entityId']
        acquisitionDate = None
        metadata = result['metadata']
        for field in metadata:
            if field['fieldName'] == "Acquisition Date":
                acquisitionDate = field['value']
                break
        cloudCover = result['cloudCover']
        publishDate = result['publishDate']
        startDate = result['temporalCoverage']['startDate']
        endDate = result['temporalCoverage']['endDate']
        spatialCoverage = result['spatialCoverage']['coordinates'][0]
        spatialBounds = result['spatialBounds']['coordinates'][0]
        polygon_geom = Polygon([(x[0], x[1]) for x in spatialCoverage])
        new_row = {'entity_id': entityId, 'acquisition_date': acquisitionDate, 'publish_date': publishDate, 'start_date': startDate,
                   'end_date': endDate, 'cloud_cover': cloudCover, 'spatial_bounds': spatialBounds, 'spatial_coverage': spatialCoverage,
                  'geometry': polygon_geom}        
        scene_metadata.append(new_row)
    gdf = gpd.GeoDataFrame(scene_metadata, crs="EPSG:4326")
    
    # convert date-like cols to date cols
    date_cols = ['acquisition_date', 'publish_date', 'start_date', 'end_date']
    for date_col in date_cols:
        gdf[date_col] = gdf[date_col].apply(lambda x: None if x == "Unknown" else x)
        gdf[date_col] = gdf[date_col].apply(lambda x: datetime.fromisoformat(x.split('.')[0]) if x is not None else x)
    
    # create year col
    gdf['start_year'] = gdf.start_date.apply(lambda x: x.year)
    gdf['end_year'] = gdf.end_date.apply(lambda x: x.year)
    
    return gdf

def get_area(shp):
    # get area of shp in km^2
    return round(shp.geometry.to_crs("EPSG:3395").map(lambda p: p.area / 10**6).iloc[0], 6)
        
def intersection_stats(shp, scene):
    intersection = gpd.overlay(shp, scene, how='intersection')
    if len(intersection) == 0:
        return {'area': 0, 'shp_percent': 0}
    
    # get area of intersection in km^2
    intersect_area = get_area(intersection)
    
    # get area of original shp in km^2
    shp_area = get_area(shp)
    
    # compute percentage of intersection of shp
    percentage = (intersect_area / shp_area) * 100
    
    return {'area': intersect_area, 'shp_percent': percentage}
    

def filter_scenes(scenes, shp_data):
    # total number of counties that could be captured by 
    total_geoms = len(shp_data)
    
        
def get_sceneIds(scenes):
    """
    Parses scene data to return list of scene ids.
    
    Parameters
    ----------
    scenes : object, required
        Output from search_scenes().
        
    
    Output
    ------
    scene ids : list
    
    """
    sceneIds = []
    for result in scenes['results']:
        sceneIds.append(result['entityId'])
    return sceneIds

def download_scenes(apiKey, sceneIds, label, dataset = "landsat_ot_c2_l2"):
    """
    Downloads scenes.
    
    Parameters
    ----------
    apiKey : str, required
        Valid API key.
        
    scenes : object, required
        Scenes you wish to download. Returned from search_scenes().
        
    label : str, required
        Label for your download request.
        
    dataset : str, optional (default is 'landsat_ot_c2_l2')
        Must be the dataset the scenes are from. 
        
    Output
    ------
    Paths to downloaded files : list
    """
        
    # download options
    payload = {
        'datasetName': dataset,
        'entityIds': sceneIds,
    }
    
    downloadOptions = sendRequest(SERVICE_URL + "download-options", payload, apiKey)
    
    # aggregate list of available products
    downloads = []
    for product in downloadOptions:
        # make sure the product is available for this scene
        if product['available'] == True:
            downloads.append({'entityId': product['entityId'],
                             'productId': product['id']})
            
    if downloads:
        requestedDownloadsCount = len(downloads)
        print("Number of Requested Downloads: {requestedDownloadsCount}".format(requestedDownloadsCount = requestedDownloadsCount))
        print("Downloading Now...")
        payload = {
            'downloads': downloads,
            'label': label
        }
        requestResults = sendRequest(SERVICE_URL + "download-request", payload, apiKey)
        if requestResults['preparingDownloads'] != None and len(requestResults['preparingDownloads']) > 0:
            payload = {'label': label}
            downloadUrls = sendRequest(SERVICE_URL + "download-retrieve", payload, apiKey)
            downloadIds = []
            for download in downloadUrls['available']:
                downloadIds.append(download['downloadId'])
                
            for download in downloadUrls['requested']:
                downloadIds.append(download['downloadId'])
                
            while len(downloadIds) < requestedDownloadsCount:
                preparingDownloads = requestedDownloadsCount - len(downloadIds)
                print('\n', preparingDownloads, "download(s) are not yet available. Waiting for 30 seconds.\n")
                time.sleep(30)
                print("Trying to retrieve data.\n")
                downloadUrls = sendRequest(SERVICE_URL + "download-retrieve", payload, apiKey)
                for download in downloadUrls['available']:
                    if download['downloadId'] not in downloadIds:
                        downloadIds.append(download['downloadId'])
        else:
            # get all available downloads
            # search requested downloads to get metadata
            payload = {
                'label': label
            }
            download_search = sendRequest(SERVICE_URL + "download-search", payload, apiKey)
            download_metadata = {}
            for search_result in download_search:
                entityId = search_result['entityId']
                displayId = search_result['displayId']
                downloadId = search_result['downloadId']
                download_metadata[downloadId] = {'entityId': entityId, 'displayId': displayId}
                
            files = []
            for download in requestResults['availableDownloads']:
                url = download['url']
                downloadId = download['downloadId']
                filename = download_file(url)
                print("{filename} Downloaded Successfully.".format(filename = filename))
                filedata = {
                    'entityId': download_metadata[downloadId]['entityId'],
                    'displayId': download_metadata[downloadId]['displayId'],
                    'filename': filename
                }
                files.append(filedata)
                
            print("\nAll files have been downloaded.\n")
            return files
        
    else:
        print("No available products.")
        
        

In [239]:
# log in to retrieve API key
config = dotenv_values('.env')
apiKey = login(config['USGS_USERNAME'], config['USGS_PASSWORD'])

Login Successful




In [253]:
DATASET_NAME = "landsat_ot_c2_l2"

In [254]:
# define temporal filter start and end constants
SEARCH_START = "2016-06-00"
SEARCH_END = "2020-09-30"

In [137]:
def get_intersection_stats(shp, metadata):
    area = []
    shp_percent = []
    for i in range(len(metadata)):
        stats = intersection_stats(shp, metadata.iloc[[i]])
        area.append(stats['area'])
        shp_percent.append(stats['shp_percent'])
    metadata['area'] = area
    metadata['shp_percent'] = shp_percent
    return metadata

In [160]:
def filter_scenes(metadata, shp):
    # filter scenes with 2 constraints:
    # 1. each year is represented (or number of years represented is maximized)
    # 2. as much of shp is covered as possible
    
    to_keep = [] # list of entity ids of scenes to keep 
    
    years = metadata.start_year.value_counts().index.tolist()
    years.sort()
    # iterate over each year 
    for year in years:
        metadata_yr = metadata[metadata.start_year == year]
        
        # sort by shp_percentage
        metadata_yr = metadata_yr.sort_values(by=["shp_percent", "cloud_cover"], ascending=False)
        
        shp_left = shp # keep track of what part of shp hasn't been covered yet
        
        # iterate over scenes in year, starting with the scene that has the largest intersection with shp
        for i in range(len(metadata_yr)):
            scene = metadata_yr.iloc[[i]].to_crs(shp.crs) # load scene and convert to crs of shp
            intersection = gpd.overlay(shp_left, scene, how='intersection')
            stats = intersection_stats(shp_left, scene)
            intersected = len(intersection) > 0
            if stats['shp_percent'] == 100.0:
                # add scene to keep 
                to_keep.append(scene.iloc[0]['entity_id'])
                break
                
            elif intersected:
                # add scene to keep
                to_keep.append(scene.iloc[0]['entity_id'])
                
                # update shp_left
                shp_left = intersection
                    
    # filter metadata to entity ids in to_keep and return
    return metadata[metadata.entity_id.isin(to_keep)]

In [162]:
# get data for Charlottesville
cville_scenes = search_scenes(apiKey, cville_bounds, SEARCH_START, SEARCH_END)
cville_scene_metadata = get_scene_metadata(cville_scenes)

# convert metadata to crs of cville spatial data
cville_scene_metadata = cville_scene_metadata.to_crs(cville_shp['counties'].crs)
cville_scene_metadata = get_intersection_stats(cville_shp['counties'].dissolve(), cville_scene_metadata)

# filter
cville_scene_metadata_f = filter_scenes(cville_scene_metadata, cville_shp['counties'].dissolve())

print("Filtered down to {num_scenes} Scene(s).".format(num_scenes = len(cville_scene_metadata_f)))

Searching Scenes...
Found 49 Scene(s).
Filtered down to 27 Scene(s).


In [164]:
# search and filter scenes for the Eastern Shore
easternShore_scenes = search_scenes(apiKey, easternShore_bounds, SEARCH_START, SEARCH_END)
easternShore_scene_metadata = get_scene_metadata(easternShore_scenes)

# convert metadata to crs of eastern shore spatial data
easternShore_scene_metadata = easternShore_scene_metadata.to_crs(easternShore_shp['counties'].crs)
easternShore_scene_metadata = get_intersection_stats(easternShore_shp['counties'].dissolve(), easternShore_scene_metadata)

# filter
easternShore_scene_metadata_f = filter_scenes(easternShore_scene_metadata, easternShore_shp['counties'].dissolve())
print("Filtered down to {num_scenes} Scene(s).".format(num_scenes = len(easternShore_scene_metadata_f)))

Searching Scenes...
Found 58 Scene(s).
Filtered down to 5 Scene(s).


In [259]:
# download cville data (just for 2020)
cville_filedata = download_scenes(
    apiKey,
    cville_scene_metadata_f[cville_scene_metadata_f.start_year == 2020].entity_id.tolist(),
    'cville_summer_2020'
)

Number of Requested Downloads: 7
LC08_L2SP_015034_20200713_20200912_02_T1.tar Downloaded Successfully.
LC08_L2SP_016033_20200906_20200918_02_T1.tar Downloaded Successfully.
LC08_L2SP_016033_20200922_20201005_02_T1.tar Downloaded Successfully.
LC08_L2SP_016034_20200704_20200913_02_T1.tar Downloaded Successfully.
LC08_L2SP_016034_20200720_20210330_02_T1.tar Downloaded Successfully.
LC08_L2SP_016034_20200906_20200918_02_T1.tar Downloaded Successfully.
LC08_L2SP_016034_20200922_20201005_02_T1.tar Downloaded Successfully.

All files have been downloaded.



In [263]:
cville_files = [x['filename'] for x in cville_filedata]
cville_files

['LC08_L2SP_015034_20200713_20200912_02_T1.tar',
 'LC08_L2SP_016033_20200906_20200918_02_T1.tar',
 'LC08_L2SP_016033_20200922_20201005_02_T1.tar',
 'LC08_L2SP_016034_20200704_20200913_02_T1.tar',
 'LC08_L2SP_016034_20200720_20210330_02_T1.tar',
 'LC08_L2SP_016034_20200906_20200918_02_T1.tar',
 'LC08_L2SP_016034_20200922_20201005_02_T1.tar']

In [264]:
easternShore_filedata = download_scenes(
    apiKey, 
    easternShore_scene_metadata_f[easternShore_scene_metadata_f.start_year == 2020].entity_id.tolist(),
    'easternShore_summer_2020'
)

Number of Requested Downloads: 1
LC08_L2SP_014034_20200722_20200911_02_T1.tar Downloaded Successfully.

All files have been downloaded.



In [266]:
easternShore_files = [x['filename'] for x in easternShore_filedata]
easternShore_files

['LC08_L2SP_014034_20200722_20200911_02_T1.tar']

In [267]:
# logout of API
# if successful, apiKey is now invalid
logout(apiKey)

Logged Out




In [175]:
# make directories where data will be extracted
os.mkdir("landsat8_c2_l2_data")
os.chdir("landsat8_c2_l2_data")
os.mkdir("cville")
os.mkdir("easternShore")
os.chdir("..")
os.getcwd()

'/Users/chasedawson/dev/uva_equity_center/climate_equity'

In [176]:
import tarfile

def extract_tar(tar, extract_to):
    """
    Extracts tar file to specified location.
    
    Parameters
    ----------
    tar : str, required
        Path to tar file that will be extracted.
        
    extract_to : str, required
        Path to folder in which the tar file will be extracted.
        
    Output
    ------
    None
    """
    print("Extracting", tar, "...")
    my_tar = tarfile.open(tar)
    my_tar.extractall(extract_to)
    my_tar.close()
    print("Done.")

In [177]:
# define paths for extracted tar files for both geographic regions
# of interest
cville_tar_path = "./landsat8_c2_l2_data/cville"
easternShore_tar_path = "./landsat8_c2_l2_data/easternShore"

In [178]:
# extract cville files and easternShore files
for tar in cville_files:
    extract_tar(tar, cville_tar_path)
    
for tar in easternShore_files:
    extract_tar(tar, easternShore_tar_path)

Extracting LC08_L2SP_016034_20200922_20201005_02_T1.tar ...
Done.
Extracting LC08_L2SP_016034_20200906_20200918_02_T1.tar ...
Done.
Extracting LC08_L2SP_016034_20200720_20210330_02_T1.tar ...
Done.
Extracting LC08_L2SP_016034_20200704_20200913_02_T1.tar ...
Done.
Extracting LC08_L2SP_016033_20200922_20201005_02_T1.tar ...
Done.
Extracting LC08_L2SP_016033_20200906_20200918_02_T1.tar ...
Done.
Extracting LC08_L2SP_015034_20200713_20200912_02_T1.tar ...
Done.
Extracting LC08_L2SP_014034_20200722_20200911_02_T1.tar ...
Done.


In [268]:
# delete original tar files
for tar in cville_files:
    os.remove(tar)
    
for tar in easternShore_files:
    os.remove(tar)

In [269]:
def remove_tar_endings(files):
    """
    Removes .tar endings from a list of filenames.
    
    Parameters
    ----------
    files : list, required
        List of tar filenames.
        
    Output
    ------
    list of filenames with .tar endings removed : list
    
    """
    reformatted_names = []
    for file in files:
        reformatted_names.append(file.split('.')[0])
    return reformatted_names

In [270]:
# remove .tar endings
cville_files = remove_tar_endings(cville_files)
easternShore_files = remove_tar_endings(easternShore_files)

## Clip Raster Data with Shapefile

In [182]:
# filename ending for raster temperature data
ST_ENDING = "_ST_B10.TIF"

In [183]:
def convert_crs(shp_data, raster_src):
    for key in shp_data:
        shp_data[key] = shp_data[key].to_crs(raster_src.crs)
    return shp_data

In [184]:
# make clipped dirs in both cville and easternShore
os.mkdir("landsat8_c2_l2_data/cville/clipped")
os.mkdir("landsat8_c2_l2_data/easternShore/clipped")

In [185]:
def clip_rasters(files, shp_data, basename):
    """
    Clip set of rasters given shapefiles. Saves these files in
    'landsat8_c2_l2_data/basename/clipped'. With the geographic region and
    spatial resolution encoded in the name.
    
    Parameters
    ----------
    files : list, required
        List of raster file base names to be clipped.
    
    shp_data : dict, required
        Geometry data for given region. A dictionary containing geometries for different spatial resolutions.
        
    basename : str, required
        Name of geographic region. Either 'cville' or 'easternShore'.
    """
    full_boundary = shp_data['counties'].dissolve()
    for file in files:
        path = 'landsat8_c2_l2_data/{basename}/{file}{ending}'.format(basename = basename, file = file, ending = ST_ENDING)
        with rio.open(path) as src:
            full_boundary = full_boundary.to_crs(src.crs)
        raster_data = rxr.open_rasterio(path, masked=True).squeeze()
        raster_clipped = raster_data.rio.clip(full_boundary.geometry.apply(mapping), full_boundary.crs)
        # export clipped raster
        new_filename = "{file}_ST_B10_{basename}_clipped.TIF".format(basename = basename, file = file)
        raster_clipped.rio.to_raster(os.path.join("landsat8_c2_l2_data", basename, "clipped", new_filename))

In [186]:
# clip and save rasters for cville and eastern shore
clip_rasters(cville_files, cville_shp, "cville")
clip_rasters(easternShore_files, easternShore_shp, "easternShore")

In [187]:
import numpy as np
from matplotlib import cm
from rasterio.plot import plotting_extent

def plot_clipped_raster(file, shp, basename):
    path = 'landsat8_c2_l2_data/{basename}/clipped/{file}_ST_B10_{basename}_clipped.TIF'.format(basename = basename, file = file)
    with rio.open(path) as src:
        st_data = src.read()
        nodata = src.nodata
        
        # change type of st_data to float64 to include nan
        st_data = st_data.astype('float64')
        st_data[st_data == nodata] = np.nan
        
        shp = shp.to_crs(src.crs)
        f, ax = plt.subplots()
        # ax.imshow(st_data[0], cm.inferno)
        ep.plot_bands(st_data, ax=ax, extent=plotting_extent(src), cmap=cm.inferno)
        shp.boundary.plot(ax=ax)
        plt.show()

# plot clipped rasters and shp files
def plot_clipped_rasters(files, shp_data, basename):
    for file in files:
        path = 'landsat8_c2_l2_data/{basename}/clipped/{file}_ST_B10_{basename}_clipped.TIF'.format(basename = basename, file = file)
        with rio.open(path) as src:
            st_data = src.read()
            nodata = src.nodata
            
            # change type of st_data to float64 to include nan
            st_data = st_data.astype('float64')
            st_data[st_data == nodata] = np.nan
            
            for key in shp_data:
                shp = shp_data[key]
                shp = shp.to_crs(src.crs)
                f, ax = plt.subplots()
                ax.imshow(st_data[0], cm.inferno)
                shp.boundary.plot(ax=ax)
                ax.set_title('{basename} {key}'.format(basename = basename, key = key))
                plt.show()

## Zonal Stats

In [227]:
from rasterstats import zonal_stats
import pandas as pd

# read in clipped files
def compute_zonal_stats(files, shp_data, basename):
    stats = pd.DataFrame()
    for file in files:
        print(file)
        path = 'landsat8_c2_l2_data/{basename}/clipped/{file}_ST_B10_{basename}_clipped.TIF'.format(basename = basename, file = file)
        with rio.open(path) as src:
            affine = src.transform
            nodata = src.profile['nodata']
            raster_data = src.read(1)
            for key in shp_data:
                gdf = shp_data[key]
                gdf = gdf.to_crs(src.crs)
                # plot_clipped_raster(file, gdf, basename)
                for i in range(len(gdf)):
                    geoId = gdf.iloc[i].GEOID
                    geom_data = gdf[gdf.GEOID == geoId]
                    geom_stats = pd.DataFrame(zonal_stats(geom_data, raster_data, affine=affine, stats = ['min','max','median','mean','nodata'], nodata=nodata))
                    geom_stats['spatial_unit'] = key
                    geom_stats['GEOID'] = geoId
                    geom_stats['file'] = '{file}_ST_B10_{basename}_clipped'.format(file = file, basename = basename)
                    stats = pd.concat([stats, geom_stats])
    return stats

In [231]:
cville_stats = compute_zonal_stats(cville_files, {key: cville_shp[key] for key in ['counties', 'tracts', 'blkgps']}, 'cville')
easternShore_stats = compute_zonal_stats(easternShore_files, {key: easternShore_shp[key] for key in ['counties', 'tracts', 'blkgps']}, 'easternShore')

LC08_L2SP_016034_20200922_20201005_02_T1
LC08_L2SP_016034_20200906_20200918_02_T1
LC08_L2SP_016034_20200720_20210330_02_T1
LC08_L2SP_016034_20200704_20200913_02_T1
LC08_L2SP_016033_20200922_20201005_02_T1
LC08_L2SP_016033_20200906_20200918_02_T1
LC08_L2SP_015034_20200713_20200912_02_T1
LC08_L2SP_014034_20200722_20200911_02_T1


/Users/chasedawson/opt/anaconda3/envs/earth-analytics-python/lib/python3.8/site-packages/rasterstats/main.py:260: UserWarning: Warning: converting a masked element to nan.
  feature_stats['nodata'] = float((featmasked == fsrc.nodata).sum())


In [233]:
def rescale(stats):
    MULTIPLICATIVE_SCALE_FACTOR = 0.00341802
    ADDITIVE_SCALE_FACTOR = 149
    cols_to_rescale = ["min", "max", "mean", "median"]
    for col in cols_to_rescale:
        stats[col] = stats[col].apply(lambda x: x * MULTIPLICATIVE_SCALE_FACTOR + ADDITIVE_SCALE_FACTOR if type(x) is float else x)
        stats[col] = stats[col].apply(lambda x: to_fahrenheit(x) if type(x) is float else x)
    return stats

def to_fahrenheit(k):
    return (k - 273.15) * (9/5) + 32
    

In [234]:
cville_stats = rescale(cville_stats)
easternShore_stats = rescale(easternShore_stats)

In [279]:
def merge_data(stats, metadata, filedata):
    entity_to_filename = {x['entityId']: x['filename'] for x in filedata}
    print(entity_to_filename)
    print([x['entityId'] for x in filedata])
    metadata = metadata[['entity_id', 'start_date', 'end_date', 'start_year', 'end_year', 'cloud_cover', 'area', 'shp_percent']]
    metadata['file'] = metadata.entity_id.apply(lambda x: entity_to_filename[x].split('.')[0])
    stats = stats.join(metadata, on="file")
    return stats

In [ ]:
cville_stats = merge_data(cville_stats, cville_scene_metadata_f, cville_filedata)
easternShore_stats = merge_data(easternShore_stats, easternShore_scene_metadata, easternShore_filedata)

In [281]:
# break into spatial units and save
spatial_units = ["blkgps", "tracts", "counties"]
for unit in spatial_units:
    cville_stats[cville_stats.spatial_unit == unit].to_csv('landsat8_cville_{unit}.csv'.format(unit = unit))
    easternShore_stats[easternShore_stats.spatial_unit == unit].to_csv('landsat8_easternShore_{unit}.csv'.format(unit = unit))

In [ ]:
# References
# [Downloading Files From Web Using Python](https://www.tutorialspoint.com/downloading-files-from-web-using-python)
# [Jupyter Tips and Tricks](https://chrieke.medium.com/jupyter-tips-and-tricks-994fdddb2057)
# [USGS/EROS Inventory Service Documentation (Machine-to-Machine) API](https://m2m.cr.usgs.gov/api/docs/json/#section-overview)
# [How are files extracted from a tar file using Python?](https://www.tutorialspoint.com/How-are-files-extracted-from-a-tar-file-using-Python)
# [How do I use a scale factor with Landsat Level-2 science products?](https://www.usgs.gov/faqs/how-do-i-use-a-scale-factor-landsat-level-2-science-products?qt-news_science_products=0#qt-news_science_products)